https://colab.research.google.com/drive/14Yc_J2hcF2guShJ5bvmG3f1O774xIbuA?usp=sharing


In [34]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
#Baseball Field Image
Img_1_Path = "/content/drive/MyDrive/Course/11002/Computer Vision/HW1/material/1.jpg"
Img_2_Path = "/content/drive/MyDrive/Course/11002/Computer Vision/HW1/material/2.jpg"
Img_3_Path = "/content/drive/MyDrive/Course/11002/Computer Vision/HW1/material/3.jpg"

#Save KRT Image
Img_Result_1_Path = "/content/drive/MyDrive/Course/11002/Computer Vision/HW1/material/Result_1.jpg"
Img_Result_2_Path = "/content/drive/MyDrive/Course/11002/Computer Vision/HW1/material/Result_2.jpg"
Img_Result_3_Path = "/content/drive/MyDrive/Course/11002/Computer Vision/HW1/material/Result_3.jpg"

#Camera Model KRT Parameter
KRT_1_Path = '/content/drive/MyDrive/Course/11002/Computer Vision/HW1/material/1.txt'
KRT_2_Path = '/content/drive/MyDrive/Course/11002/Computer Vision/HW1/material/2.txt'
KRT_3_Path = '/content/drive/MyDrive/Course/11002/Computer Vision/HW1/material/3.txt'

#Baseball Trajectory xWorld Position 
TrajectoryXYZ_Path = '/content/drive/MyDrive/Course/11002/Computer Vision/HW1/material/trajectory.xyz'

# 讀取 KRT 檔案, 並劃分為K矩陣與RT矩陣

In [36]:
def GetKRTParaFromFile(FilePath):
    K_Float_List, RT_Float_List = [], []
    with open(FilePath) as f:
        Rows = f.readlines()
        for Row in Rows:
            # 判別目前讀取到的矩陣資料列
            if Row == 'K\n':
                Matrix = 'K'
            elif Row == 'RT\n':
                Matrix = 'RT'
            # 基本正規劃處理
            if Matrix =='K' and Row != 'K\n':
                K_Float_List.append([float(i) for i in Row.replace('\n','').split(' ')]) 
            elif Matrix =='RT' and Row != 'RT\n':
                RT_Float_List.append([float(i) for i in Row.replace('\n','').split(' ')]) 

    return np.array(K_Float_List),np.array(RT_Float_List) 

# 讀取棒球彈道 TrajectoryXYZ 檔案, 轉換為轉換為矩陣，並需要增加固定數為 1 ,限縮相片距離

In [37]:

def GetTrajectoryXYZParaFromFile(FilePath):
    Trajectory_List = []
    with open(FilePath) as f:
        Rows = f.readlines()
        for Row in Rows:
            X_World = (Row.replace('\n','').split(' '))
            X_World.append('1')                          ## 設置X_World 當中 參數為 1 , 可限縮相片距離
            Trajectory_List.append([float(i) for i in X_World]) 
    return np.array(Trajectory_List)

# 計算 Camera Model 公式 x = K[R|T]X , 使用矩陣相乘

In [38]:

def KRT_MultipleMatrix(K_Matrix_Src, RT_Matrix_Src, Trajectory_Matrix_Src):
    print("MultipleMatrix")
    Trajectory_Image_Matrix = []
    for row in Trajectory_Matrix_Src:
        Mul_Matrix = K_Matrix_Src.dot(RT_Matrix_Src).dot(row)
        Mul_Matrix = Mul_Matrix/Mul_Matrix[2]
        Mul_Matrix=np.delete(Mul_Matrix,(2), axis = 0)
        Trajectory_Image_Matrix.append(Mul_Matrix)
    return Trajectory_Image_Matrix


# 顯示影像於頁面中, 並整合處理流程

- ShowImageWithBaseball 顯示棒球於相片中
- KRT_Func 資料處理流程資料處理流程
  1. GetKRTParaFromFile           = 處理 KRT 資料
  1. GetTrajectoryXYZParaFromFile = 處理 TrajectoryXYZ 棒球座標矩陣
  1. KRT_MultipleMatrix             處理 x = K[R|T]X 公式
  1. ShowImageWithBaseball        ＝ 顯示計算結果
---



In [39]:
def ShowImageWithBaseball(Ori_Image, Result_Image, Trajectory_Image_Matrix):

    cv_ori = cv2.imread(Ori_Image)

    Trajectory_Image_Matrix=np.float32(Trajectory_Image_Matrix)
    for point in Trajectory_Image_Matrix:
        BaseBallPosition=cv2.circle(cv_ori,(int(point[0]),int(point[1])),15,(128,255,128))
    cv2_imshow(BaseBallPosition)
    cv2.imwrite(Result_Image, BaseBallPosition)
    

def KRT_Func(KRT_Path,TrajectoryXYZ_Path,Image_Ori,Image_Result):
    K_Matrix, RT_Matrix = GetKRTParaFromFile(KRT_Path)
    Trajectory_Matrix = GetTrajectoryXYZParaFromFile(TrajectoryXYZ_Path)
    Trajectory_Image_Matrix = KRT_MultipleMatrix(K_Matrix,RT_Matrix,Trajectory_Matrix)
    ShowImageWithBaseball(Image_Ori, Image_Result, Trajectory_Image_Matrix)    

# 執行三張圖表

In [40]:
if __name__ == '__main__':

    #ShowImageWithBaseball()
    KRT_Func(KRT_1_Path,TrajectoryXYZ_Path,Img_1_Path, Img_Result_1_Path)
    KRT_Func(KRT_2_Path,TrajectoryXYZ_Path,Img_2_Path, Img_Result_2_Path)
    KRT_Func(KRT_3_Path,TrajectoryXYZ_Path,Img_3_Path, Img_Result_3_Path)
    #cv2.waitKey(0)



Output hidden; open in https://colab.research.google.com to view.